# Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|ستون|توضیحات|
|:------:|:---:|
|<code>Type</code>| نوع حیوان|
|<code>Age</code>| سن حیوان خانگی (به ماه) هنگام لیست شدن|
|<code>Breed1</code>|نژاد اصلی حیوان خانگی|
|<code>Gender</code>|جنسیت حیوان خانگی|
|<code>Color1</code>|رنگ اصلی حیوان خانگی|
|<code>Color2</code>|رنگ دوم حیوان خانگی (در صورت وجود)|
|<code>MaturitySize</code>|اندازه|
|<code>FurLength</code>|طول خز|
|<code>Vaccinated</code>|وضعیت واکسینه شدن|
|<code>Sterilized</code>|وضعیت انگل‌زدایی شدن|
|<code>Health</code>|وضعیت سلامتی|
|<code>Fee</code>|هزینه|
|<code>Description</code>|توضیحات|
|<code>PhotoAmt</code>|تعداد عکس بارگذاری شده|
|<code>AdoptionSpeed</code>|سرعت پذیرش (توضیح این اعداد در ادامه آمده است)|

<br>

|توضیحات|سرعت پذیرش|
|:------:|:---:|
|حیوان خانگی در همان روزی که لیست شده به سرپرستی پذیرفته شد.|<code>0</code>|
|	حیوان خانگی بین ۱ تا ۷ روز (هفته اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>1</code>|
|حیوان خانگی بین ۸ تا ۳۰ روز (ماه اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>2</code>|
|حیوان خانگی بین ۳۱ تا ۹۰ روز (ماه دوم و سوم) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>3</code>|
|پس از ۱۰۰ روز لیست شدن، حیوان خانگی به سرپرستی پذیرفته نشد.|<code>4</code>|

</font>
</div>
</center>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data set reading

In [21]:
# Load the train data
train = pd.read_csv('/content/drive/MyDrive/Quera/new_home_tensorflow/data/petfinder_train.csv')

In [22]:
train.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           10537 non-null  object
 1   Age            10537 non-null  int64 
 2   Breed1         10537 non-null  object
 3   Gender         10537 non-null  object
 4   Color1         10537 non-null  object
 5   Color2         10537 non-null  object
 6   MaturitySize   10537 non-null  object
 7   FurLength      10537 non-null  object
 8   Vaccinated     10537 non-null  object
 9   Sterilized     10537 non-null  object
 10  Health         10537 non-null  object
 11  Fee            10537 non-null  int64 
 12  Description    10528 non-null  object
 13  PhotoAmt       10537 non-null  int64 
 14  AdoptionSpeed  10537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.2+ MB


# Preprocessing

In [24]:
mapping={0:1,1:1,2:1,3:1,4:0}

train['Target'] = train['AdoptionSpeed'].map(mapping)

In [25]:
train.drop(columns = ['AdoptionSpeed', 'Description'], inplace = True)

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          10537 non-null  object
 1   Age           10537 non-null  int64 
 2   Breed1        10537 non-null  object
 3   Gender        10537 non-null  object
 4   Color1        10537 non-null  object
 5   Color2        10537 non-null  object
 6   MaturitySize  10537 non-null  object
 7   FurLength     10537 non-null  object
 8   Vaccinated    10537 non-null  object
 9   Sterilized    10537 non-null  object
 10  Health        10537 non-null  object
 11  Fee           10537 non-null  int64 
 12  PhotoAmt      10537 non-null  int64 
 13  Target        10537 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.1+ MB


In [27]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

column: Type unique_values: 2
column: Breed1 unique_values: 165
column: Gender unique_values: 2
column: Color1 unique_values: 7
column: Color2 unique_values: 7
column: MaturitySize unique_values: 3
column: FurLength unique_values: 3
column: Vaccinated unique_values: 3
column: Sterilized unique_values: 3
column: Health unique_values: 3


In [28]:
le = LabelEncoder()

train['MaturitySize'] = le.fit_transform(train['MaturitySize'])

train['FurLength'] = le.fit_transform(train['FurLength'])

train['Health'] = le.fit_transform(train['Health'])

In [29]:
train['Type'] = le.fit_transform(train['Type'])

train['Gender'] = le.fit_transform(train['Gender'])

In [12]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.8 MB/s eta 0:00:00


In [30]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = binary_encoder.fit_transform(train['Breed1'])

train_binary_Color1 =  binary_encoder.fit_transform(train['Color1'])

train_binary_Color2 =  binary_encoder.fit_transform(train['Color2'])

train_binary_Vaccinated =  binary_encoder.fit_transform(train['Vaccinated'])

train_binary_Sterilized =  binary_encoder.fit_transform(train['Sterilized'])

In [31]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Type          10537 non-null  int64
 1   Age           10537 non-null  int64
 2   Gender        10537 non-null  int64
 3   MaturitySize  10537 non-null  int64
 4   FurLength     10537 non-null  int64
 5   Health        10537 non-null  int64
 6   Fee           10537 non-null  int64
 7   PhotoAmt      10537 non-null  int64
 8   Target        10537 non-null  int64
 9   Breed1_0      10537 non-null  int64
 10  Breed1_1      10537 non-null  int64
 11  Breed1_2      10537 non-null  int64
 12  Breed1_3      10537 non-null  int64
 13  Breed1_4      10537 non-null  int64
 14  Breed1_5      10537 non-null  int64
 15  Breed1_6      10537 non-null  int64
 16  Breed1_7      10537 non-null  int64
 17  Color1_0      10537 non-null  int64
 18  Color1_1      10537 non-null  int64
 19  Color1_2      10537 non-n

In [32]:
X= train.drop('Target',axis=1)
y= train.Target

In [33]:
for column in X.columns:
  mean = train[column].mean()
  std = train[column].std()
  train[column] = (train[column]- mean)/std

train.head()

,Type,Age,Gender,MaturitySize,FurLength,Health,Fee,PhotoAmt,Target,Breed1_0,...,Color1_0,Color1_1,Color1_2,Color2_0,Color2_1,Color2_2,Vaccinated_0,Vaccinated_1,Sterilized_0,Sterilized_1
0,-1.153719,-0.449427,1.136673,1.629890,0.790516,-0.194365,0.949788,-0.830874,1,-0.073746,...,-0.521041,-0.861486,0.761850,-0.520158,-1.286339,0.637933,-1.151487,0.386892,-0.730867,0.360165
1,-1.153719,-0.552656,1.136673,-0.226574,-0.845318,-0.194365,-0.297960,-0.512200,1,-0.073746,...,-0.521041,-0.861486,0.761850,-0.520158,0.777326,-1.567414,0.868360,-2.584453,1.368109,-2.776244
2,0.866680,-0.552656,1.136673,-0.226574,-0.845318,-0.194365,-0.297960,1.081167,1,-0.073746,...,-0.521041,1.160675,-1.312469,-0.520158,-1.286339,0.637933,0.868360,0.386892,-0.730867,0.360165
3,0.866680,-0.397813,-0.879677,-0.226574,0.790516,-0.194365,1.573661,1.399841,1,-0.073746,...,-0.521041,-0.861486,0.761850,-0.520158,0.777326,-1.567414,0.868360,0.386892,-0.730867,0.360165
4,0.866680,-0.552656,1.136673,-0.226574,0.790516,-0.194365,-0.297960,-0.193527,1,-0.073746,...,-0.521041,-0.861486,0.761850,-0.520158,0.777326,0.637933,-1.151487,0.386892,-0.730867,0.360165


In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=43)

print('Train examples:', len(X_train), len(y_train))
print('Validation examples:', len(X_valid), len(y_valid))

Train examples: 9483 9483
Validation examples: 1054 1054


# Modeling with Keras(low level)

In [36]:
model1 = keras.Sequential()
model1.add(keras.layers.Input(shape=(26,)))
model1.add(keras.layers.Dense(5000, activation='relu'))
model1.add(keras.layers.Dense(1000, activation='relu'))
model1.add(keras.layers.Dense(500, activation='relu'))
model1.add(keras.layers.Dense(1, activation='sigmoid'))

In [37]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 5000)                │         135,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1000)                │       5,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 500)                 │         500,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             501 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,637,001 (21.50 MB)

 Trainable params: 5,637,001 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model1.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy'])

In [39]:
epochs = 10
BATCH_SIZE = 128

history = model1.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 15s 167ms/step - accuracy: 0.6702 - loss: 1.3369 - val_accuracy: 0.6869 - val_loss: 0.6031
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 188ms/step - accuracy: 0.7301 - loss: 0.5471 - val_accuracy: 0.7021 - val_loss: 0.5483
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 177ms/step - accuracy: 0.7308 - loss: 0.5421 - val_accuracy: 0.7334 - val_loss: 0.5333
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 191ms/step - accuracy: 0.7404 - loss: 0.5287 - val_accuracy: 0.7381 - val_loss: 0.5566
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 15s 206ms/step - accuracy: 0.7560 - loss: 0.5231 - val_accuracy: 0.7287 - val_loss: 0.5266
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 183ms/step - accuracy: 0.7378 - loss: 0.5185 - val_accuracy: 0.7391 - val_loss: 0.5176
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 178ms/step - accuracy: 0.7461 - loss: 0.5154 - val_accuracy: 0.7277 - val_loss: 0.5144
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 184ms/step - accuracy: 0.7518 - loss: 0.5097 - val_accu

# Modeling with Tensorflow(high level)

In [40]:
train = train.astype(np.float32)

In [41]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid,y_valid))

In [42]:
class CustomModel(tf.Module):

    def __init__(self, input_features):

        initializer = tf.initializers.GlorotUniform()
        self.w1 = tf.Variable(initializer(shape=[input_features,5000]), name = 'w1')
        self.bias1 = tf.Variable(tf.ones([5000]), name = 'bias1')

        self.w2 = tf.Variable(initializer(shape=[5000,1000]),name='w2')
        self.bias2 = tf.Variable(tf.ones([1000]), name='bias2')

        self.w3 = tf.Variable(initializer(shape=[1000,500]),name='w3')
        self.bias3 = tf.Variable(tf.ones([500]), name='bias3')

        self.w_output = tf.Variable(initializer(shape=[500,1]),name='w_output')
        self.bias_output = tf.Variable(tf.ones([1]),name='bias_output')


    def forward(self, x):
        x = tf.cast(x, tf.float32)
        self.first = tf.matmul(x, self.w1) + self.bias1
        self.first = tf.nn.relu(self.first,name='1st')
        self.second = tf.matmul(self.first, self.w2) + self.bias2
        self.second = tf.nn.relu(self.second,name='2nd')
        self.third = tf.matmul(self.second, self.w3) + self.bias3
        self.third = tf.nn.relu(self.third,name='3nd')
        self.output = tf.matmul(self.third, self.w_output) + self.bias_output
        self.output = tf.nn.sigmoid(self.output,name='output')
        return self.output

    def predict(self, x):
        batched_dataset = x.batch(len(x))
        X_batch = next(iter(batched_dataset))
        x = self.forward(X_batch)
        return x

    @tf.function
    def fit_one_batch(self, inputs, targets, optimizer, loss_fn):
        with tf.GradientTape() as tape:
            predictions = self.forward(inputs)
            loss = loss_fn(targets, predictions)

        gradients = tape.gradient(loss, self.trainable_variables)
        optimizer.apply_gradients(zip(gradients,self.trainable_variables))

        return loss


    def fit(self, dataset, valid_dataset,  epochs, BATCH_SIZE,
            optimizer = tf.optimizers.Adam(), loss_fn = tf.losses.BinaryCrossentropy()):

        batched_validation_dataset = valid_dataset.batch(len(valid_dataset))
        X_valid, y_valid = next(iter(batched_validation_dataset))

        batched_dataset = dataset.batch(BATCH_SIZE)


        for epoch in range(epochs):
            for X_batch, y_batch in batched_dataset:
                loss = self.fit_one_batch(X_batch, y_batch, optimizer, loss_fn)

            val_predictions = self.forward(X_valid)
            val_loss = loss_fn(y_valid, val_predictions)

            val_f1 = f1_score(y_valid.numpy(), val_predictions.numpy() > 0.5, average = 'weighted')

            print(f'Epoch {epoch+1}, Loss: {loss}, Validation Loss: {val_loss.numpy()}, Validation F1 score: {val_f1}')


In [43]:
input_shape = 26 #train.shape[1] - 1
model = CustomModel(input_shape)
EPOCHS=64
BATCH_SIZE=128
model.fit(train_dataset, valid_dataset, epochs=EPOCHS, BATCH_SIZE=BATCH_SIZE)

Epoch 1, Loss: 0.30230817198753357, Validation Loss: 0.8491266965866089, Validation F1 score: 0.6168023236459269
Epoch 2, Loss: 0.25087547302246094, Validation Loss: 0.6544537544250488, Validation F1 score: 0.6163391869231547
Epoch 3, Loss: 0.2826974391937256, Validation Loss: 0.5252665281295776, Validation F1 score: 0.6935528914859459
Epoch 4, Loss: 0.26406148076057434, Validation Loss: 0.523002028465271, Validation F1 score: 0.6864104820158375
Epoch 5, Loss: 0.24383804202079773, Validation Loss: 0.5779050588607788, Validation F1 score: 0.6176207041956567
Epoch 6, Loss: 0.24395577609539032, Validation Loss: 0.5159646272659302, Validation F1 score: 0.708548738589817
Epoch 7, Loss: 0.24097727239131927, Validation Loss: 0.5196183919906616, Validation F1 score: 0.6672250508727795
Epoch 8, Loss: 0.24230074882507324, Validation Loss: 0.5265376567840576, Validation F1 score: 0.6480947317506849
Epoch 9, Loss: 0.23925498127937317, Validation Loss: 0.5210810899734497, Validation F1 score: 0.663